<a href="https://colab.research.google.com/github/karthikvarma1/Karthik-Resume/blob/gh-pages/Hackathon_Challenge_2__DataGeeks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**DATA GEEKS**

1) Vinay Kumar Reddy Gaddam 


2) Shamshuddin Dudekula


3) Karthik Varma Keerthipati




**Challenge2 - non-STEM assessment**

**OBJECTIVE**  : To prepare a model to grade non-STEM examinations using LSTM.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


**Imported Libraries**

In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import nltk
from nltk.corpus import stopwords
import re
from gensim.models import Word2Vec
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K
from tqdm import tqdm

from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import cohen_kappa_score

**Loading Data**

In [ ]:

df = pd.read_csv("/data_essay.csv")

FileNotFoundError: ignored

In [ ]:
df['essay_set'].unique()  

array([1, 2, 3, 4, 5, 6, 7, 8])

In [ ]:
q1= df.loc[df['essay_set'] == 1]
q2= df.loc[df['essay_set'] == 2]
q3= df.loc[df['essay_set'] == 3]
q4= df.loc[df['essay_set'] == 4]
q5= df.loc[df['essay_set'] == 5]
q6= df.loc[df['essay_set'] == 6]
q7= df.loc[df['essay_set'] == 7]
q8= df.loc[df['essay_set'] == 8]

In [ ]:
print("Scoring range of q1",sorted(q1.score.unique()))
print("Scoring range of q2",sorted(q2.score.unique()))
print("Scoring range of q3",sorted(q3.score.unique()))
print("Scoring range of q4",sorted(q4.score.unique()))
print("Scoring range of q5",sorted(q5.score.unique()))
print("Scoring range of q6",sorted(q6.score.unique()))
print("Scoring range of q7",sorted(q7.score.unique()))
print("Scoring range of q8",sorted(q8.score.unique()))


Scoring range of q1 [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
Scoring range of q2 [1, 2, 3, 4, 5, 6]
Scoring range of q3 [0, 1, 2, 3]
Scoring range of q4 [0, 1, 2, 3]
Scoring range of q5 [0, 1, 2, 3, 4]
Scoring range of q6 [0, 1, 2, 3, 4]
Scoring range of q7 [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
Scoring range of q8 [10, 15, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 55, 60]


##**Data Processing**

As we know data processing steps include tokenizing, lower casing, stop words, stemming.
We have performed required steps accordingly.

In [ ]:
q1t = q1['essay']
q2t = q2['essay']
q3t = q3['essay']
q4t = q4['essay']
q5t = q5['essay']
q6t = q6['essay']
q7t = q7['essay']
q8t = q8['essay']

In [ ]:

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

nltk.download('stopwords')
stop_words = set(stopwords.words("english"))
new = set(['well', 'using', 'us', 'recommend', 'use', ])
stop_words =stop_words.union(new)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:

def preprocess(df):
  text = []
# tqdm is for printing the status bar
  for sentance in tqdm(df.values):  
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub('\W+','', sentance)
    #sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stop_words)
    text.append(sentance.strip())
  return text

In [ ]:
p1 =  preprocess(q1t)
p2 =  preprocess(q2t)
p3 =  preprocess(q3t)
p4 =  preprocess(q4t)
p5 =  preprocess(q5t)
p6 =  preprocess(q6t)
p7 =  preprocess(q7t)
p8 =  preprocess(q8t)


100%|██████████| 723/723 [00:00<00:00, 2098.13it/s]


In [ ]:
q1["P_essay"] = p1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
q2["P_essay"] = p2
q3["P_essay"] = p3
q4["P_essay"] = p4
q5["P_essay"] = p5
q6["P_essay"] = p6
q7["P_essay"] = p7
q8["P_essay"] = p8

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
X= q1
y = X.score

##**Data Modelling**

For tokenization we have used feature vector , as it is a collection of text.

In [ ]:

def makeFeatureVec(words, model, num_features):
    """Make Feature Vector from the words list of an Essay."""
    featureVec = np.zeros((num_features,),dtype="float32")
    num_words = 0.
    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set:
            num_words += 1
            featureVec = np.add(featureVec,model[word])        
    featureVec = np.divide(featureVec,num_words)
    return featureVec

def getAvgFeatureVecs(essays, model, num_features):
    """Main function to generate the word vectors for word2vec model."""
    counter = 0
    essayFeatureVecs = np.zeros((len(essays),num_features),dtype="float32")
    for essay in essays:
        essayFeatureVecs[counter] = makeFeatureVec(essay, model, num_features)
        counter = counter + 1
    return essayFeatureVecs

Long Short-Term Memory (LSTM) networks are a type of recurrent neural network capable of learning order dependence in sequence prediction problems. This is a behavior required in complex problem domains like machine translation, speech recognition, and more. LSTMs are a complex area of deep learning.

Here we have used LSTM to get our required result.

In [ ]:


def get_model(n):
    """Define the model."""
    model = Sequential()
    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))
    model.add(Dense(n, activation='softmax'))

    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['accuracy','mae'])
    model.summary()

    return model

In [ ]:
n = len(q1.score.unique())

In [ ]:

cv = KFold(n_splits = 5, shuffle = True)
results = []
y_pred_list = []
count = 1
for traincv, testcv in cv.split(X):
    print("\n--------Fold {}--------\n".format(count))
    X_test, X_train, y_test, y_train = X.iloc[testcv], X.iloc[traincv], y.iloc[testcv], y.iloc[traincv]
    train_essays = list(X_train['P_essay'].values)
    test_essays = list(X_test['P_essay'].values)
    # Initializing variables for word2vec model.
    num_features = 300 
    min_word_count = 40
    num_workers = 4
    context = 10
    downsampling = 1e-3

    print("Training Word2Vec Model...")
    model = Word2Vec(train_essays, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)

    model.init_sims(replace=True)
    model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)
    trainDataVecs = getAvgFeatureVecs(train_essays, model, num_features)
    testDataVecs = getAvgFeatureVecs(test_essays, model, num_features )
    
    trainDataVecs = np.array(trainDataVecs)
    testDataVecs = np.array(testDataVecs)
    # Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
    trainDataVecs = np.reshape(trainDataVecs, (trainDataVecs.shape[0], 1, trainDataVecs.shape[1]))
    testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))
    
    lstm_model = get_model(n)
    lstm_model.fit(trainDataVecs, y_train, batch_size=64, epochs=50)
    #lstm_model.load_weights('./model_weights/final_lstm.h5')
    y_pred = lstm_model.predict(testDataVecs)
    
    # Save any one of the 5 models.
    if count == 5:
         lstm_model.save('./final_lstm.h5')
    
    # Round y_pred to the nearest integer.
    y_pred = np.around(y_pred)
    
    # Evaluate the model on the evaluation metric. "Quadratic mean averaged Kappa"
    result = cohen_kappa_score(y_test.values,y_pred,weights='quadratic')
    from sklearn.metrics import accuracy_score
    acc = accuracy_score(y_test.values,y_pred)
    print("acc Score: {}".format(acc))
    print("Kappa Score: {}".format(result))
    results.append(result)

    count += 1


--------Fold 1--------

Training Word2Vec Model...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 300)            721200    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                93440     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 814,705
Trainable params: 814,705
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
23/23 [==============================] - 25s 16ms/step - loss: 66.7215 - accuracy: 0.0000e+00 - mae: 7.9935
Epoch 2/50
23/23 [==============================] - 0s 17ms/step - loss: 17.7606 - accuracy: 0.0000e+00 - mae: 3.7882
Epoch 3/50


In [ ]:
print("Average Kappa score after a 5-fold cross validation: ",np.around(np.array(results).mean(),decimals=4))

Average Kappa score after a 5-fold cross validation:  0.1489


##**kappa Score**

Cohens Kappa is used to distinguish between reliability and validity. It does really good for multi-class problems. It is the quantitative measure of  reliability for two raters, rating for the same thing, and corrected for how often that the raters might agree by chance.